# <b><span style='color:#2563eb'>00 | </span>Analiza Eksploracyjna Danych</b>

## <span style='color:#2563eb'>🔷 | <b></span>Dostępne pliki</b>

In [ ]:
# Set auto reload after making changes
%load_ext autoreload
%autoreload 2

#PATH = 'kaggle/input/predict-energy-behavior-of-prosumers'
PATH = 'data/'

import os
for dirname, _, filenames in os.walk(PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## <span style='color:#2563eb'>🔷 | <b></span>Opis danych</b>

### ✨ <b>train.csv</b>

|Nazwa kolumny|Opis|
|-|-|
|county|Identyfikator hrabsta|
|is_business|Czy prosumer jest biznesem|
|product_type|Rodzaje: Combined(0), Fixed(1), General service(2), Spot(3)|
|target|Konsumpcja lub produkcja dla danego segmentu na godzinę. Segment = `county` + `is_business` + `product_type`|
|is_consumption|Określenie, czy target jest produkcją, czy konsumpcją|
|datetime|Czas - strefa estońska (UTC+2)|
|data_block_id|Wszystkie rekordy z tym samym `data_block_id` są dostępne w tym samym czasie prognozy|
|row_id|Unikalny identyfiaktor wiersza|
|prediction_unit_id| Unikalny identyfikator dla `county` + `is_business` + `product_type`|


### ✨ <b>gas_prices.csv</b>

|Nazwa kolumny|Opis|
|-|-|
|origin_date|Data udostępnienia prognoz dnia następnego|
|forecast_date|Data prognozy|
|lowest_price_per_mwh|Najniższa cena gazu naturalnego dla dnia predykcji (€/MWh)|
|highest_price_per_mwh|Najwyższa cena gazu naturalnego dla dnia predykcji (€/MWh)|
|data_block_id|


### ✨ <b>client.csv</b>

|Nazwa kolumny|Opis|
|-|-|
|county|Identyfikator hrabsta|
|eic_count|Zagregowana liczba punktów zużycia (European Identifier Code)|
|installed_capacity|Zainstalowana moc paneli w kW|
|is_business|Czy prosumer jest biznesem|
|date||
|data_block_id|Wszystkie rekordy z tym samym `data_block_id` są dostępne w tym samym czasie prognozy|


### ✨ <b>electricity_prices.csv</b>

|Nazwa kolumny|Opis|
|-|-|
|origin_date||
|forecast_date||
|euros_per_mwh|Cena prądu dla dnia predykcji (€/MWh)|
|data_block_id|Wszystkie rekordy z tym samym `data_block_id` są dostępne w tym samym czasie prognozy|


### ✨ <b>historical_weather.csv</b>

|Nazwa kolumny|Opis|
|-|-|
|datetime||
|temperature|Temperatura 2 metry powyżej gruntu w stopniach Celsjusza|
|dewpoint|Temperatura punktu rosy 2 metry powyżej gruntu w stopniach Celsjusza - temp., w której może rozpocząć się proces skraplania gazu|
|rain|Deszcz z wielkoskalowych systemów pogodowych z poprzedniej godziny w milimetrach|
|snowfall|Opady śniegu w ciągu ostatniej godziny w centymetrach|
|surface_pressure|Ciśnienie powietrza na powierzchni w hPa|
|cloudcover_low/mid/high/total|Procent nieba przysłoniętego chmurami w przedziałach wysokości 0-3km, 3-8km, 8+, oraz całościowo|
|windspeed_10m|Prędkość wiatru na wysokości 10 metrów nad ziemią w m/s|
|winddirection_10m|Kierunek wiatru na wysokości 10 metrów nad ziemią w stopniach|
|shortwave_radiation|Globalne poziome napromieniowanie w watogodzinach na metr kwadratowy|
|direct_solar_rediation|Bezpośrednie promieniowanie słoneczne docierające do powierzchni w płaszczyźnie prostopadłej do kierunku słońca, skumulowane w ciągu poprzedniej godziny ($Wh/m^2$)|
|diffuse_radiation|Rozproszone promieniowanie słoneczne w ($Wh/m^2$)|
|latitude/longitude|Współrzędne stacji pogodowej|
|data_block_id|Wszystkie rekordy z tym samym `data_block_id` są dostępne w tym samym czasie prognozy|


### ✨ <b>forecast_weather.csv</b>

|Nazwa kolumny|Opis|
|-|-|
|latitude/longitude|Współrzędne prognozy pogody|
|origin_datetime|Czas generowania prognozy pogody|
|hours_ahead|Liczba godzin pomiędzy generowaniem prognozy i pogodą. Każda prognoza obejmuje 48 godzin|
|temperature|Temperatura 2 metry powyżej gruntu w stopniach Celsjusza|
|dewpoint|Temperatura punktu rosy 2 metry powyżej gruntu w stopniach Celsjusza - temp., w której może rozpocząć się proces skraplania gazu|
|cloudcover_low/mid/high/total|Procent nieba przysłoniętego chmurami w przedziałach wysokości 0-2km, 2-6km, 6+, oraz całościowo|
|10_metre_u/v_wind_component|Składowa wschodnia i północna prędkości wiatru mierzona 10m powyżej powierzchni w m/s|
|data_block_id|Wszystkie rekordy z tym samym `data_block_id` są dostępne w tym samym czasie prognozy |
|forecast_datetime|Znacznik czasu przewidywania pogody. Możliwy do wyznaczenia na podstawie `origin_datetime` i `hours_ahead`|
|direct_solar_rediation|Bezpośrednie promieniowanie słoneczne docierające do powierzchni w płaszczyźnie prostopadłej do kierunku słońca, skumulowane w ciągu poprzedniej godziny ($Wh/m^2$)|
|surface_solar_radiation_downwards|Promieniowanie słoneczne, zarówno bezpośrednie, jak i rozproszone, docierające do płaszczyzny poziomej na powierzchni Ziemi ($Wh/m^2$)|
|snowfall|Opady śniegu w ciągu ostatniej godziny przeliczone na metry wody|
|total_precipitation|Ciecz pochodząca z deszczu i śniegu, która opadła na powierzchnię w przeciągu godziny - metry wody|



## <span style='color:#2563eb'>🔷 | <b></span>Analiza product_type</b>

### ✨ <b>Combined</b>

Umowa łączona w kontekście energii zazwyczaj odnosi się do umowy, która zawiera wiele elementów lub cech z różnych rodzajów umów. Może obejmować połączenie stałych i zmiennych cen, różnych poziomów usług lub kombinacji źródeł energii.

> Cena: 

### ✨ <b>Fixed</b>

W umowie stałej warunki umowy określają stałą, niezmienną cenę za dostarczoną energię przez ustalony okres. Zapewnia to konsumentowi stabilność i przewidywalność, niezależnie od wahań na rynku.

> Cena: Na podstawie zużytych MWh

### ✨ <b>General service</b>

Ogólna umowie o świadczenie usług to standardowa, często regulowana umowa, która gwarantuje konsumentom podstawowy poziom usług energetycznych. Może mieć z góry określone warunki, a struktura cen może być stała lub zmienna.

> Cena: Ustalona cena za ograniczone zużycie + cena rynkowa za zużycie ponad limit

### ✨ <b>Spot</b>

Kontakt spotowy polega na zakupie lub sprzedaży energii po aktualnej cenie rynkowej. Cena jest ustalana na podstawie warunków podaży i popytu w czasie rzeczywistym. Jest realtywnie podatny na wahania cen.

> Cena: Cena rynkowa za zużycie

## <span style='color:#2563eb'>🔷 | <b></span>Import bibliotek</b>

In [ ]:
import json
import numpy as np
import pandas as pd
import plotly.express as px

# Map library
import folium

# Charts
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Wrote myself
from source.CustomPlot import CustomPlot
from source.Utils import SplitDateColumn, DescribeData

# Set formatting for dataframes
pd.options.display.float_format = '{:,.3f}'.format

## <span style='color:#2563eb'>🔷 | <b></span>Dane o prowincjach</b>

In [ ]:
county_name = json.load(open(os.path.join(PATH, 'county_id_to_name_map.json')))

county_name = {int(k):str(v) for k,v in county_name.items()}

# Estonian county data
county_locations = pd.DataFrame({
    "county": ["HARJUMAA", "HIIUMAA", "IDA-VIRUMAA", "JÄRVAMAA", "JÕGEVAMAA", 
               "LÄÄNE-VIRUMAA", "LÄÄNEMAA", "PÄRNUMAA", "PÕLVAMAA", "RAPLAMAA", 
               "SAAREMAA", "TARTUMAA", "VALGAMAA", "VILJANDIMAA", "VÕRUMAA"],
    "latitude": [59.351142, 58.918082, 59.228971, 58.897934, 58.727941, 
                 59.267897, 58.975935, 58.448793, 58.089925, 58.924451, 
                 58.414075, 58.394168, 57.933466, 58.316916, 57.765485],
    "longitude": [24.725384, 22.586403, 27.406654, 25.623048, 26.413961, 
                  26.363968, 23.772451, 24.526469, 27.101149, 24.619842, 
                  22.525137, 26.747568, 26.191360, 25.595130, 27.025669]
})

## <span style='color:#2563eb'>🔷 | <b></span>Współrzędne punktów pogodowych</b>

In [ ]:
weather_station = pd.read_csv(os.path.join(PATH, 'weather_station_to_county_mapping.csv'),
                   dtype={'county_name': 'str',
                          'longitude': 'float64',
                          'latitude': 'float64',
                          'county': 'float64'})

weather_station.dropna(subset='county', inplace=True)
weather_station.drop(columns=['county_name'], inplace=True)
weather_station['county'] = weather_station['county'].astype('int')
weather_station[['latitude', 'longitude']] = weather_station[['latitude', 'longitude']].astype(float).round(1)

## <span style='color:#2563eb'>🔷 | <b></span>Analiza poszczególnych zbiorów danych</b>

### ✨ <b>gas_prices</b>

In [ ]:
gas_prices = pd.read_csv(os.path.join(PATH, 'gas_prices.csv'),
                   dtype={'lowest_price_per_mwh': 'float64',
                          'highest_price_per_mwh': 'float64',
                          'data_block_id': 'int64'},
                   parse_dates=['forecast_date', 'origin_date'])

In [ ]:
DescribeData(gas_prices)

In [ ]:
fig = px.line(
    gas_prices.rename(columns={'lowest_price_per_mwh' : 'Najniższa cena za MWh', 'highest_price_per_mwh' : 'Najwyższa cena za MWh'}),
    x='forecast_date',
    y=['Najniższa cena za MWh', 'Najwyższa cena za MWh'], 
    title='Ceny gazu zimnego w czasie',
    labels={
        'forecast_date' : 'Data prognozy',
        'value' : 'Wartość'
    },
    height=500,
    width=1000)

fig.update_xaxes(tickformat="%d-%m-%Y")
fig.update_layout(legend_title_text='Zmienne')

### ✨ <b>electricity_prices</b>

In [ ]:
electricity_prices = pd.read_csv(os.path.join(PATH, 'electricity_prices.csv'),
                   dtype={'euros_per_mwh': 'float64',
                          'data_block_id': 'int64'},
                   parse_dates=['forecast_date', 'origin_date'])

In [ ]:
electricity_prices = SplitDateColumn(electricity_prices, 'forecast_date', suffix='forecast')

DescribeData(electricity_prices)

In [ ]:
data = electricity_prices.groupby(['forecast_date'])['euros_per_mwh'].mean().reset_index(name='euros_per_mwh')

fig = px.line(
    data.rename(columns={'euros_per_mwh' : 'Euro za MWh'}),
    x='forecast_date',
    y=['Euro za MWh'], 
    title='Ceny energii elektrycznej w czasie',
    labels={
        'value' : 'Wartość',
        'forecast_date' : 'Data'
    },
    height=500,
    width=1000)
fig.update_layout(legend_title_text='Zmienne')
fig.update_xaxes(tickformat="%d-%m-%Y")
fig.show()

### ✨ <b>historical_weather</b>

In [ ]:
historical_weather = pd.read_csv(os.path.join(PATH, 'historical_weather.csv'),
                dtype={'temperature': 'float64',
                        'dewpoint': 'float64',
                        'rain': 'float64',
                        'snowfall': 'float64',
                        'surface_pressure': 'float64',
                        'cloudcover_total': 'int16',
                        'cloudcover_low': 'int16',
                        'cloudcover_mid': 'int16',
                        'cloudcover_high': 'int16',
                        'winddirection_10m': 'int16',
                        'shortwave_radiation': 'float64',
                        'direct_solar_radiation' : 'float64',
                        'diffuse_radiation': 'float64',
                        'latitude': 'float64',
                        'longitude' : 'float64',
                        'data_block_id' : 'int64'},

                parse_dates=['datetime'])

In [ ]:
DescribeData(historical_weather)

### ✨ <b>forecast_weather</b>

In [ ]:
forecast_weather = pd.read_csv(os.path.join(PATH, 'forecast_weather.csv'),
                dtype={'temperature': 'float64',
                        'dewpoint': 'float64',
                        'total_precipitation': 'float64',
                        'snowfall': 'float64',
                        'cloudcover_total': 'float64',
                        'cloudcover_low': 'float64',
                        'cloudcover_mid': 'float64',
                        'cloudcover_high': 'float64',
                        '10_metre_u_wind_component': 'float64',
                        '10_metre_v_wind_component': 'float64',
                        'direct_solar_radiation' : 'float64',
                        'surface_solar_radiation_downwards': 'float64',
                        'latitude': 'float64',
                        'longitude' : 'float64',
                        'data_block_id' : 'int64',
                        'hours_ahead': 'int16'},

                parse_dates=['origin_datetime', 'forecast_datetime'])

#### 💡 <b>Wstępne przetworzenie danych</b>

In [ ]:
forecast_weather = forecast_weather.rename(columns = {'forecast_datetime': 'datetime'})
forecast_weather.drop(columns = 'origin_datetime', inplace=True)
forecast_weather['datetime'] = forecast_weather['datetime'].dt.tz_convert('Europe/Brussels').dt.tz_localize(None)

# Map to weather locations
forecast_weather[['latitude', 'longitude']] = forecast_weather[['latitude', 'longitude']].astype(float).round(1)
forecast_weather = forecast_weather.merge(weather_station, how='left', on=['latitude', 'longitude'])

# Some weather locations are outside any county
forecast_weather.dropna(subset='county', inplace=True)

forecast_weather['county'] = forecast_weather['county'].astype(int)

In [ ]:
# Some county have many weather locations
forecast_weather = forecast_weather.groupby(by=['datetime', 'county', 'data_block_id']).mean().reset_index()
forecast_weather = SplitDateColumn(forecast_weather, column='datetime')

In [ ]:
DescribeData(forecast_weather)

### ✨ <b>train</b>

In [ ]:
train = pd.read_csv(os.path.join(PATH, 'train.csv'),
                dtype={ 'county': 'int16',
                        'is_business': 'boolean',
                        'product_type': 'int8',
                        'target': 'float64',
                        'is_consumption': 'boolean',
                        'data_block_id' : 'int64',
                        'row_id' : 'int16',
                        'prediction_unit_id' : 'int16' },

                parse_dates=['datetime'])

In [ ]:

train[(train.target > 11255) & (train.is_consumption == False)]

In [ ]:
DescribeData(train)

In [ ]:
fig = px.histogram(
    train, 
    x='target', 
    title='Histogram przewidywanej wartości',
    labels={'target' : 'Przewidywana wartość'},
    height=500,
    width=1000)

fig.update_layout(yaxis_title='Liczebność')
fig.show()

In [ ]:
types = {0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}
data = train[['product_type', 'is_business']].copy()
data['product_type'].replace(types, inplace=True)

fig=px.histogram(
    data,
    x='product_type',
    title='Histogram typów rozliczeń prosumentów',
    labels={'product_type' : 'Typ'},
    height=500,
    width=1000,
    color='is_business')

fig.update_layout(yaxis_title='Liczebność')
fig.show()

In [ ]:
columns = ['is_business', 'is_consumption', 'product_type']
fig = make_subplots(1, len(columns), specs=[[{'type':'domain'} for _ in range(len(columns))]], subplot_titles=columns)

for i, column in enumerate(columns):
    lbs, counts = np.unique(train[column].values, return_counts=True)
    
    if column == 'product_type':
        lbs = list(map(lambda type: types[type], lbs))
        
    fig.add_trace(go.Pie(labels=lbs, values=counts, scalegroup='one'), 1, i + 1)

fig.update_layout(title_text='Rozkład wartości wybranych atrybutów', showlegend=False)
fig.update_traces(textinfo='label')
fig.update_layout(
    width=1000,
    height=500)

In [ ]:
split_column = 'is_business'
columns = ['is_consumption', 'product_type']


for i, split_value in enumerate(train[split_column].unique().tolist()):
    fig = make_subplots(1, len(columns), specs=[[{'type':'domain'} for _ in range(len(columns))]], subplot_titles=columns)

    data = train.query(f'{split_column} == {split_value}')
    for i, column in enumerate(columns):
        lbs, counts = np.unique(data[column].values, return_counts=True)

        if column == 'product_type':
            lbs = list(map(lambda type: types[type], lbs))
    
        fig.add_trace(go.Pie(labels=lbs, values=counts, scalegroup='one'), 1, i + 1)

    fig.update_layout(title_text=f'Rozkład wartości wybranych atrybutów, gdzie {split_column} = {split_value}', showlegend=False, height=500, width=1000)
    fig.update_traces(textinfo='label')
    fig.show()

### ✨ <b>client</b>

In [ ]:
client = pd.read_csv(os.path.join(PATH, 'client.csv'),
                dtype={ 'county': 'int16',
                        'is_business': 'boolean',
                        'product_type': 'int8',
                        'eic_count': 'float64',
                        'installed_capacity': 'float64',
                        'data_block_id' : 'int16'},

                parse_dates=['date'])

In [ ]:
DescribeData(client)

In [ ]:
fig = px.histogram(
    client,
    x='installed_capacity',
    title='Rozkład zainstalowanej mocy',
    color='is_business',
    labels={'installed_capacity' : 'Zainstalowana moc'},
    height=500,
    width=1000)

fig.update_layout(yaxis_title='Liczebność')
fig.show()

In [ ]:
data = client.copy()
data = SplitDateColumn(data, 'date')[['county', 'date_year', 'installed_capacity']]
data = data.groupby(['county', 'date_year']).max().reset_index()
data['county'] = data['county'].replace(county_name)


fig = go.Figure()
for year in [2021, 2022, 2023]:
    year_data = data[data.date_year == year]

    fig.add_trace(go.Bar(
        x = year_data['county'],
        y = year_data['installed_capacity'],
        name = year
    ))


fig.update_layout(title_text='Zainstalowana moc powiatów Estonii w kolejnych latach')
fig.update_layout(yaxis_title='Moc [kW]')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
types = {0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}

data = client.groupby(['date', 'is_business'])['installed_capacity'].mean().reset_index(name='installed_capacity')

fig = px.line(
    data,
    x='date',
    y='installed_capacity',
    facet_col='is_business',
    title=f'Zainstalowana moc w czasie',
    labels={
        'installed_capacity' : 'Zainstalowana moc',
        'date' : 'Czas'},
    width=1000,
    height=500)

fig.update_xaxes(tickformat="%d-%m-%Y")
fig.show()

## <span style='color:#2563eb'>🔷 | <b></span>Produkcja/Konsumpcja energii</b>

In [ ]:
types = {0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}
train['date'] = train['datetime'].dt.date

# Only energy production data
#data = train.query('is_consumption == 0')

data = train.groupby(['date', 'product_type', 'is_business', 'is_consumption'])['target'].mean().reset_index(name='target')

    
fig = px.line(
    data,
    x='date',
    y=['target'],
    facet_col='is_business',
    facet_row='product_type',
    color='is_consumption',
    title=f'Konsumpcja/Produkcja energii w czasie w zależności od grupy prosumentów',
    labels={
        'value' : 'Wartość',
        'date' : 'Czas'
    },
    height=800,
    width=1000)

fig.update_xaxes(tickformat="%d-%m-%Y")
fig.show()

## <span style='color:#2563eb'>🔷 | <b></span>Rozmieszczenie punktów pogodowych</b>

In [ ]:
# Create a map with Estonia focused
map_ = folium.Map([58, 25], zoom_start=7)

location_columns = ["latitude", "longitude"]
weather_places = historical_weather.groupby(location_columns).size().reset_index()
forecast_places = forecast_weather.groupby(location_columns).size().reset_index()

# Add historical weather
for index, row in weather_places.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=3, color='orange').add_child(folium.Popup(str(row['latitude']) + ' x ' + str(row['longitude']))).add_to(map_)

# Add forecast locations
for index, row in forecast_places.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=1, color='blue').add_child(folium.Popup(str(row['latitude']) + ' x ' + str(row['longitude']))).add_to(map_)

# Add counties locations
for index, row in county_locations.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=4, color='red').add_child(folium.Popup(row['county'])).add_to(map_)

map_

## <span style='color:#2563eb'>🔷 | <b></span>Pogoda w poszczególnie dni</b>

In [ ]:
analyzedColumns = ['temperature', 'snowfall', 'surface_solar_radiation_downwards', 'production', 'consumption']

# Get mean from each county
data = forecast_weather.groupby(by=['datetime_date', 'datetime_time']).mean().reset_index()

consumption = train.query('is_consumption == True')[['datetime', 'target']].groupby(by=['datetime']).mean().reset_index().rename(columns={'target' : 'consumption'})
production = train.query('is_consumption == False')[['datetime', 'target']].groupby(by=['datetime']).mean().reset_index().rename(columns={'target' : 'production'})

data = data.merge(consumption, how='left', on=['datetime']).merge(production, how='left', on=['datetime'])

CustomPlot(data[['datetime', 'datetime_date'] + analyzedColumns], 'datetime_date').Slider(title='Weather forecast for').Scatter(normalize=True).Show()

## <span style='color:#2563eb'>🔷 | <b></span>Działanie block_id</b>

In [ ]:
gas_prices.query('data_block_id == 174')

In [ ]:
def findDataBlockRange(df: pd.DataFrame, name: str, column='datetime'):
    df_grouped = df[[column, 'data_block_id']].groupby('data_block_id')
    df_min = df_grouped.min().reset_index('data_block_id').rename(columns={column : 'Początek'})
    df_max = df_grouped.max().reset_index('data_block_id').rename(columns={column : 'Koniec'})
    df_count = df_grouped.count().reset_index('data_block_id').rename(columns={column : 'Liczebność'})
    df_data = df_min.merge(df_max, on=['data_block_id']).merge(df_count, on=['data_block_id'])
    df_data['Zbiór danych'] = [name] * len(df_data)

    return df_data

hw_data = findDataBlockRange(historical_weather, 'Historyczna pogoda')
fw_data = findDataBlockRange(forecast_weather, 'Prognoza pogody')
train_data = findDataBlockRange(train, 'Produkcja/Konsumpcja')
elect_data = findDataBlockRange(electricity_prices, 'Ceny energii elektrycznej', column='forecast_date')
gas_data = findDataBlockRange(gas_prices, 'Ceny gazu ziemnego', column='forecast_date')
client_data = findDataBlockRange(client, 'Dane o prosumentach', column='date')

block_id_data = pd.concat([hw_data, fw_data, elect_data, gas_data, train_data, client_data], axis=0)

CustomPlot(block_id_data, 'data_block_id').Table().Slider('Data block id').Show()

Każdego dnia o 11:00 należy dokonywać predykcji wartości 'target' dla każdej godziny od 00:00 do 23:00 dnia następnego.

W danym bloku dostępna jest informacja o przeszłej pogodzie od poprzedniego dnia 11:00 do godziny 10 tego samego dnia, oraz 48 godzin prognozy pogody (od godziny 01:00 tego samego dnia do godziny 00:00 dwa dni w przód)

## <span style='color:#2563eb'>🔷 | <b></span>Konsumpcja/Produkcja - korelacja</b>


In [ ]:
consumption = train.query('is_consumption == True')[['target', 'date']].groupby(by=['date']).mean().reset_index().rename(columns={'target' : 'consumption'})
production = train.query('is_consumption == False')[['target', 'date']].groupby(by=['date']).mean().reset_index().rename(columns={'target' : 'production'})

data = production.merge(consumption, how='left', on=['date'])

fig = px.imshow(data[['production', 'consumption']].corr())
fig.show()

## <span style='color:#2563eb'>🔷 | <b></span>Konsumpcja/Produkcja w poszczególnych powiatach</b>

In [ ]:
consumption = train.query('is_consumption == True and is_business == False')[['datetime', 'target', 'county', 'date']].groupby(by=['date', 'county']).mean().reset_index().rename(columns={'target' : 'consumption'})
production = train.query('is_consumption == False and is_business == False')[['datetime', 'target', 'county', 'date']].groupby(by=['date', 'county']).mean().reset_index().rename(columns={'target' : 'production'})

data = production.merge(consumption, how='left', on=['county', 'datetime'])
columns = ['production', 'consumption', 'datetime', 'county']

CustomPlot(data[columns], 'county').Scatter().Slider(title='Target per county (is_business == False)', prefix='County').Show()

In [ ]:
consumption = train.query('is_consumption == True and is_business == True')[['datetime', 'target', 'county', 'date']].groupby(by=['date', 'county']).mean().reset_index().rename(columns={'target' : 'consumption'})
production = train.query('is_consumption == False and is_business == True')[['datetime', 'target', 'county', 'date']].groupby(by=['date', 'county']).mean().reset_index().rename(columns={'target' : 'production'})

data = production.merge(consumption, how='left', on=['county', 'datetime'])
columns = ['production', 'consumption', 'datetime', 'county']

CustomPlot(data[columns], 'county').Scatter().Slider(title='Target per county is_business == True', prefix='County').Show()

In [ ]:
data = train.copy()
data = SplitDateColumn(data, 'datetime')
data = train[['datetime', 'target', 'county', 'is_business', 'is_consumption', 'datetime_month', 'datetime_year']].groupby(by=['datetime_month', 'datetime_year', 'county', 'is_business', 'is_consumption']).mean().reset_index()
data.sort_values(by=['datetime', 'county'], inplace=True)
data['county'] = data['county'].replace(county_name)

fig = px.line(
    data,
    x='datetime',
    y=['target'],
    facet_col='is_business',
    facet_row='is_consumption',
    color='county',
    title=f'Konsumpcja/Produkcja energii w czasie dla powiatów (agregacja miesięczna)',
    labels={
        'value' : 'Produkcja/Konsumpcja',
        'datetime' : 'Czas'
    },
    height=800,
    width=1000)

fig.update_xaxes(tickformat="%d-%m-%Y")
fig.show()

## <span style='color:#2563eb'>🔷 | <b></span>Konsumpcja/Produkcja w poszczególnych powiatach - próba normalizacji</b>


In [ ]:
# train <- client
on = ['county', 'is_business', 'product_type', 'data_block_id']

merged_data = train.merge(client[on + ['installed_capacity']], how='left', on=on)

consumption = merged_data.query('is_consumption == True and is_business == False')[['datetime', 'target', 'county', 'date', 'installed_capacity']].groupby(by=['date', 'county']).mean().reset_index().rename(columns={'target' : 'consumption'})
production = merged_data.query('is_consumption == False and is_business == False')[['datetime', 'target', 'county', 'date', 'installed_capacity']].groupby(by=['date', 'county']).mean().reset_index().rename(columns={'target' : 'production'})

# Filling missing data
mean = consumption['installed_capacity'].mean()
consumption['installed_capacity'].fillna(mean, inplace=True)

mean = production['installed_capacity'].mean()
production['installed_capacity'].fillna(mean, inplace=True)

# Dividing target by installed_capacity
production['production'] = production['production'] / production['installed_capacity']
consumption['consumption'] = consumption['consumption'] / consumption['installed_capacity']

data = production.merge(consumption, how='left', on=['county', 'datetime'])
columns = ['production', 'consumption', 'datetime', 'county']

CustomPlot(data[columns], 'county').Scatter().Slider(title='Target per county (is_business == False)', prefix='County').Show()

In [ ]:
# train <- client
on = ['county', 'is_business', 'product_type', 'data_block_id']

merged_data = train.merge(client[on + ['installed_capacity']], how='left', on=on)

consumption = merged_data.query('is_consumption == True and is_business == True')[['datetime', 'target', 'county', 'date', 'installed_capacity']].groupby(by=['date', 'county']).mean().reset_index().rename(columns={'target' : 'consumption'})
production = merged_data.query('is_consumption == False and is_business == True')[['datetime', 'target', 'county', 'date', 'installed_capacity']].groupby(by=['date', 'county']).mean().reset_index().rename(columns={'target' : 'production'})

# Filling missing data
mean = consumption['installed_capacity'].mean()
consumption['installed_capacity'].fillna(mean, inplace=True)

mean = production['installed_capacity'].mean()
production['installed_capacity'].fillna(mean, inplace=True)

# Dividing target by installed_capacity
production['production'] = production['production'] / production['installed_capacity']
consumption['consumption'] = consumption['consumption'] / consumption['installed_capacity']

data = production.merge(consumption, how='left', on=['county', 'datetime'])
columns = ['production', 'consumption', 'datetime', 'county']

CustomPlot(data[columns], 'county').Scatter().Slider(title='Target per county (is_business == False)', prefix='County').Show()

## <span style='color:#2563eb'>🔷 | <b></span>Suma konsumpcji i produkcji - trend</b>

In [ ]:
# train <- client
on = ['county', 'is_business', 'product_type', 'data_block_id']

merged_data = train.merge(client[on + ['installed_capacity']], how='left', on=on)

consumption = merged_data.query('is_consumption == True')[['datetime', 'target', 'date']].groupby(by=['date']).mean().reset_index().rename(columns={'target' : 'consumption'})
production = merged_data.query('is_consumption == False')[['datetime', 'target', 'date']].groupby(by=['date']).mean().reset_index().rename(columns={'target' : 'production'})

data = consumption.merge(production, on=['date', 'datetime'], how='left')
data['sum'] = data['consumption'] + data['production']

fig = px.line(
    data,
    x='datetime',
    y=['sum'], 
    title='Suma konsumpcji i produkcji w czasie',
    labels={
        'value' : 'Wartość',
        'datetime' : 'Data'
    },
    height=500,
    width=1000)
fig.update_layout(legend_title_text='Zmienne')
fig.update_xaxes(tickformat="%d-%m-%Y")
fig.show()

## <span style='color:#2563eb'>🔷 | <b></span>Konsumpcja - trend</b>

In [ ]:
# train <- client
on = ['county', 'is_business', 'product_type', 'data_block_id']

consumption = SplitDateColumn(train, 'datetime').query('is_consumption == True')[['datetime', 'target', 'datetime_month', 'datetime_year']].groupby(by=['datetime_month', 'datetime_year']).mean().reset_index().rename(columns={'target' : 'consumption'})

consumption.sort_values(by=['datetime'], inplace=True)

fig = px.line(
    consumption,
    x='datetime',
    y=['consumption'], 
    title='Konsumpcja w czasie (agregacja miesięczna)',
    labels={
        'value' : 'Wartość',
        'datetime' : 'Data'
    },
    height=500,
    width=1000)
fig.update_layout(legend_title_text='Zmienne')
fig.update_xaxes(tickformat="%d-%m-%Y")
fig.show()

## <span style='color:#2563eb'>🔷 | <b></span>Różnicowanie - problem trendu</b>

In [ ]:
from skforecast.preprocessing import TimeSeriesDifferentiator

def diff(df:pd.DataFrame, column: str):
    diffenciator = TimeSeriesDifferentiator()
    diffenciator.fit(df[column].to_numpy())

    df[column] = diffenciator.transform(df[column].to_numpy())

    return df

consumption = train.query('is_consumption == True and is_business == False')[['datetime', 'target', 'date']].groupby(by=['date']).mean().reset_index().rename(columns={'target' : 'consumption'})
production = train.query('is_consumption == False and is_business == False')[['datetime', 'target', 'date']].groupby(by=['date']).mean().reset_index().rename(columns={'target' : 'production'})

data = production.merge(consumption, how='left', on=['datetime'])

# Differentiation
data = diff(data, 'production')
data = diff(data, 'consumption')

fig = px.line(
    data,
    x='datetime',
    y=['consumption', 'production'], 
    title='Konsumpcja/Produkcja - różnicowanie',
    labels={
        'value' : 'Wartość',
        'datetime' : 'Data'
    },
    height=500,
    width=1000)
fig.update_layout(legend_title_text='Zmienne')
fig.update_xaxes(tickformat="%d-%m-%Y")
fig.show()